In [1]:
import requests as r
import pandas as pd
from decimal import Decimal
import pandas_gbq

import sys
sys.path.append("../helper_functions")
import duneapi_utils as d
import pandas_utils as p
sys.path.pop()

import time
import os
import dotenv
from google.cloud import bigquery
from google.oauth2 import service_account

dotenv.load_dotenv()

# Set the environment variable to the path of your service account key file
print(os.environ["IS_RUNNING_LOCAL"])
if os.environ["IS_RUNNING_LOCAL"] == True: #GH Action was weird with this, so forcing the datatype here
        print('running local')
        # os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.getenv("PATH_TO_BQ_CREDS")
        pandas_gbq.context.credentials = service_account.Credentials.from_service_account_file(os.getenv("PATH_TO_BQ_CREDS"))
else:
        print('not running local')
        # os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.getenv("BQ_APPLICATION_CREDENTIALS")
        pandas_gbq.context.credentials = os.getenv("BQ_APPLICATION_CREDENTIALS")

# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.getenv("BQ_APPLICATION_CREDENTIALS")
# pandas_gbq.context.credentials = service_account.Credentials.from_service_account_file()
pandas_gbq.context.project = "oplabs-tools-data"
# Initialize a BigQuery client
# client = bigquery.Client()

running local


In [2]:
chain_mappings = {
        'zora': 'Zora',
        'pgn': 'Public Goods Network',
        'base': 'Base',
        'mode': 'Mode',
        'metal': 'Metal',
        'frax': 'Fraxtal'
        # Add more mappings as needed
    }

In [3]:
sql_directory = "inputs/sql/"

query_names = [
        # Must match the file name in inputs/sql
        "frax_allltime_chain_activity"
]

In [4]:
#checks
print(pandas_gbq.context.project)
dune_df = []
dune_table = 'op_gcs_allltime_chain_activity'

oplabs-tools-data


In [5]:
for qn in query_names:
        # If we can do it programmatically from UI saved queries
        # query = client.get_job(query_name)
        # Read the SQL query from file
        with open(os.path.join(sql_directory, f"{qn}.sql"), "r") as file:
                query = file.read()
        print(qn)
        table_name = qn
        
        # Execute the query
        # query_job = client.query(query)
        df = pandas_gbq.read_gbq(query)
        
        # Write to csv
        df.to_csv('outputs/chain_data/' + qn + '.csv', index=False)
        # print(df.sample(5))
        time.sleep(1)
        
        # Write to Dune
        df['chain_raw'] = df['chain']
        df['chain'] = df['chain'].replace(chain_mappings)
        dune_df.append(df)

write_df = pd.concat(dune_df)
d.write_dune_api_from_pandas(write_df, dune_table,table_description = dune_table)
write_df.to_csv('outputs/chain_data/' + dune_table + '.csv', index=False)
# Print the results

frax_allltime_chain_activity


/Users/michaelsilberling/opt/anaconda3/envs/gcp-env/lib/python3.12/site-packages/google/cloud/bigquery/table.py:2383: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


table at: dune.oplabspbc.dataset_op_gcs_allltime_chain_activity
Response status code: 200
Response content: b'{"success":true,"table_name":"op_gcs_allltime_chain_activity"}'


In [6]:
# print(dune_df)